In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 10:42:54 2020
@ date; 31 august 2021
@author: Claudia Acquistapace
@goal: plot radiosonde quantities for the case study of the 2nd february using data from l'atalante
"""

# importing necessary libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as mdates
import glob
from myFunctions import lcl
from myFunctions import f_closest
from warnings import warn
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy import interpolate
import custom_color_palette as ccp
from matplotlib import rcParams
import matplotlib.ticker as ticker

# reading radiosondes
path_RS = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/radiosondes_atalante/case_1/'
file_list_RS = [path_RS+'EUREC4A_Atalante_Vaisala-RS_L2_v3.0.0.nc', path_RS+'EUREC4A_Atalante_Meteomodem-RS_L2_v3.0.0.nc']
path_out = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/'
# +
# read radiosonde from vaisala
data_RS_vaisala = xr.open_dataset(file_list_RS[0])

# read time of the launch for all radiosondes
time_launch = pd.to_datetime(data_RS_vaisala.launch_time.values)
sounding_id = data_RS_vaisala.sounding.values

# find indeces for the cold patch
ind = np.where((time_launch > datetime(2020,2,2,0,0,0)) * (time_launch < datetime(2020,2,3,23,59,59)))[0]
start = sounding_id[ind[0]]
end = sounding_id[ind[-1]]

# selecting data_RS
data_vaisala_sel = data_RS_vaisala.sel(sounding=slice(start, end))
print('n rs vaisala', len(data_vaisala_sel))

# +
# read radiosonde from meteogram
data_RS_meteo = xr.open_dataset(file_list_RS[1])

# read time of the launch for all radiosondes
time_launch = pd.to_datetime(data_RS_meteo.launch_time.values)
sounding_id = data_RS_meteo.sounding.values

# find indeces for the cold patch
ind = np.where((time_launch > datetime(2020,2,2,0,0,0)) * (time_launch < datetime(2020,2,3,23,59,59)))[0]
start = sounding_id[ind[0]]
end = sounding_id[ind[-1]]

# selecting data_RS
data_meteo_sel = data_RS_meteo.sel(sounding=slice(start, end))
print('n rs meteomodem', len(data_meteo_sel))


# +
# merging the two selected datasets
data_patch = xr.merge([data_vaisala_sel, data_meteo_sel])
print()

# re-ordering files in temporal order
sounding_id = pd.to_datetime(data_patch.launch_time.values)

# re-ordering and saving order of indeces in ind_sorted
soundings_strings_sorted = np.sort(sounding_id)
ind_sorted = np.argsort(sounding_id)


n rs vaisala 19
n rs meteomodem 19


/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/xarray/core/alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  index = joiner(matching_indexes)


In [6]:
data_meteo_sel

<xarray.Dataset>
Dimensions:      (alt: 3100, nv: 2, sounding: 28)
Coordinates:
  * alt          (alt) int16 0 10 20 30 40 50 ... 30950 30960 30970 30980 30990
    flight_time  (sounding, alt) datetime64[ns] ...
    lat          (sounding, alt) float32 ...
    lon          (sounding, alt) float32 ...
    launch_time  (sounding) datetime64[ns] 2020-02-02T19:32:36 ... 2020-02-03...
  * sounding     (sounding) object 'Atalante__ascent__6.42_-53.68__2020020219...
Dimensions without coordinates: nv
Data variables:
    p            (sounding, alt) float32 ...
    dz           (sounding, alt) float32 ...
    ta           (sounding, alt) float32 ...
    dp           (sounding, alt) float32 ...
    wspd         (sounding, alt) float32 ...
    u            (sounding, alt) float32 ...
    v            (sounding, alt) float32 ...
    mr           (sounding, alt) float32 ...
    theta        (sounding, alt) float32 ...
    q            (sounding, alt) float32 ...
    alt_bnds     (alt, nv) int16 ...
    wdir         (sounding, alt) float32 ...
    platform     (sounding) int16 ...
    rh           (sounding, alt) float32 ...
    N_ptu        (sounding, alt) float32 ...
    N_gps        (sounding, alt) float32 ...
    m_ptu        (sounding, alt) int16 ...
    m_gps        (sounding, alt) int16 ...
    ascent_flag  (sounding) int16 ...
Attributes:
    title:            EUREC4A level 2 sounding data
    platform:         Atalante
    instrument:       Radiosonde GPSonde M10 by MeteoModem
    doi:              10.25326/137
    created_with:     batch_interpolate_soundings.py with its last modificati...
    created_on:       Tue Dec 22 22:09:50 2020
    featureType:      trajectory
    Conventions:      CF-1.7
    campaign_id:      EUREC4A
    references:       Stephan et al. (2020): Ship- and island-based atmospher...
    acknowledgement:  The MPI-M is listed as the institute of first contact. ...
    instrument_id:    Meteomodem-RS
    version:          v3.0.0

In [7]:
data_vaisala_sel

<xarray.Dataset>
Dimensions:      (alt: 3100, nv: 2, sounding: 23)
Coordinates:
  * alt          (alt) int16 0 10 20 30 40 50 ... 30950 30960 30970 30980 30990
    flight_time  (sounding, alt) datetime64[ns] ...
    lat          (sounding, alt) float32 ...
    lon          (sounding, alt) float32 ...
    launch_time  (sounding) datetime64[ns] 2020-02-02T00:44:19.583056 ... 202...
  * sounding     (sounding) object 'Atalante__ascent__8.35_-52.70__2020020200...
Dimensions without coordinates: nv
Data variables:
    p            (sounding, alt) float32 ...
    dz           (sounding, alt) float32 ...
    ta           (sounding, alt) float32 ...
    dp           (sounding, alt) float32 ...
    wspd         (sounding, alt) float32 ...
    u            (sounding, alt) float32 ...
    v            (sounding, alt) float32 ...
    mr           (sounding, alt) float32 ...
    theta        (sounding, alt) float32 ...
    q            (sounding, alt) float32 ...
    alt_bnds     (alt, nv) int16 ...
    wdir         (sounding, alt) float32 ...
    platform     (sounding) int16 ...
    rh           (sounding, alt) float32 ...
    N_ptu        (sounding, alt) float32 ...
    N_gps        (sounding, alt) float32 ...
    m_ptu        (sounding, alt) int16 ...
    m_gps        (sounding, alt) int16 ...
    ascent_flag  (sounding) int16 ...
Attributes:
    title:            EUREC4A level 2 sounding data
    platform:         Atalante
    instrument:       Radiosonde RS41-SGP by Vaisala
    doi:              10.25326/137
    created_with:     batch_interpolate_soundings.py with its last modificati...
    created_on:       Tue Dec 22 20:52:54 2020
    featureType:      trajectory
    Conventions:      CF-1.7
    campaign_id:      EUREC4A
    references:       Stephan et al. (2020): Ship- and island-based atmospher...
    acknowledgement:  The MPI-M is listed as the institute of first contact. ...
    instrument_id:    Vaisala-RS
    version:          v3.0.0

In [3]:
len(soundings_strings_sorted)

51